# Aspect-level Sentiment Classification

Reading material:
- [1] R. He, WS. Lee & D. Dahlmeier. Exploiting document knowledge for aspect-level sentiment classification. 2018. https://arxiv.org/abs/1806.04346.


Build an attention-based aspect-level sentiment classification model with biLSTM. The model includes:

- BiLSTM network that learns sentence representation from input sequences.
- Attention network that assigns attention score over a sequence of biLSTM hidden states based on aspect terms representation.
- Fully connected network that predicts sentiment label, given the representation weighted by the attention score.

Requirements:

- You shall train your model bsaed on transferring learning. That is, you need first train your model on documnet-level examples. Then the learned weights will be used to initialize aspect-level model and fine tune it on aspect-level examples.
- You shall use the alignment score function in attention network as following expression:$$f_{score}(h,t)=tanh(h^TW_a t)$$
- You shall evaluate the trained model on the provided test set and show the accuracy on test set.


In [ ]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [ ]:
import _pickle as cPickle


#Load Data

In [ ]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [ ]:
# aspect_path = '/drive/My Drive/Deep Learing Course/practice-5-data/aspect_level-sentiment/aspect_level/' 
aspect_path = '/drive/My Drive/Deep Learning/practice/Practical 5/practice-5.1-data/aspect-level/'

vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')

pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [ ]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
        
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [ ]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM, Bidirectional
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints


Using TensorFlow backend.


In [ ]:
overal_maxlen = 82
overal_maxlen_aspect = 7


#Define Attention Network Layer
- Define class for Attention Layer
- You need to finish the code for calculating the attention weights

In [ ]:
class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list
        return None

    def call(self, input_tensor, mask=None):
        x = input_tensor[0]
        aspect = input_tensor[1]
        mask = mask[0]

        aspect = K.transpose(K.dot(self.W, K.transpose(aspect)))
        aspect = K.expand_dims(aspect, axis=-2)
        aspect = K.repeat_elements(aspect, self.steps, axis=1)
        eij = K.sum(x*aspect, axis=-1)

        eij = K.tanh(eij)
        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        return a

   
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1])

In [ ]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

#Establish computation Grah for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [ ]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

##Input tensors

In [ ]:
##### Inputs #####
sentence_input = Input(shape=(overal_maxlen,), dtype='int32', name='sentence_input')
aspect_input = Input(shape=(overal_maxlen_aspect,), dtype='int32', name='aspect_input')
pretrain_input = Input(shape=(None,), dtype='int32', name='pretrain_input')

##Shared WordEmbedding layer

In [ ]:
##### construct word embedding layer #####
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')

### represent aspect as averaged word embedding ###
print ('use average term embs as aspect embedding')
# get the embedding representation for asepct terms
aspect_term_embs = word_emb(aspect_input)
# average it Using Average  
aspect_embs = Average(mask_zero=True, name='aspect_emb')(aspect_term_embs)

use average term embs as aspect embedding


In [ ]:
### sentence representation from embedding ###
# from aspect-level domain
sentence_embs = word_emb(sentence_input)
# from document-level domain
pretrain_embs = word_emb(pretrain_input)

##Shared BiLSTM layer

In [ ]:
### shared BiLSTM layer###
bilstm = Bidirectional(LSTM(300, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout), name='bilstm')

### sentence representation from embedding ###
sentence_lstm = bilstm(sentence_embs)  #sentence from aspect-level
pretrain_lstm = bilstm(pretrain_embs)  #sentence from document-level


##Attention Layer

In [ ]:
att_weights = Attention(name='att_weights')([sentence_lstm, aspect_embs])
# wtighted sum
dotor = Dot(axes = 1, name='dotor_att')
att_context = dotor([att_weights, sentence_lstm])

##Prediction Layer

In [ ]:
pretrain_avg = Average(mask_zero=True)(pretrain_lstm)
# dropout layer
sentence_output = Dropout(dropout)(att_context)
pretrain_output = Dropout(dropout)(pretrain_avg)
# dense layer
sentence_output = Dense(num_outputs, name='dense_1')(sentence_output)
pretrain_output = Dense(num_outputs, name='dense_2')(pretrain_output)
aspect_probs = Activation('softmax', name='aspect_model')(sentence_output)
doc_probs = Activation('softmax', name='pretrain_model')(pretrain_output)

#Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [ ]:
model1 = Model(inputs=pretrain_input, outputs=doc_probs)
model2 = Model(inputs=[sentence_input, aspect_input], outputs=[aspect_probs])
model1.summary()
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrain_input (InputLayer)  (None, None)              0         
_________________________________________________________________
word_emb (Embedding)         multiple                  3000900   
_________________________________________________________________
bilstm (Bidirectional)       multiple                  1442400   
_________________________________________________________________
average_1 (Average)          (None, 600)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1803      
_________________________________________________________________
pretrain_model (Activation)  (None, 3)                 0   

#Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

##Train on document-level data

In [ ]:
import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter_doc):
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter_doc]
            for train_batch in train_batches:
                yield train_batch
                
    history = model.fit_generator(train_gen(), steps_per_epoch=train_steps_epoch, \
                                  epochs = 10, callbacks = earlystop_callbacks)

In [ ]:
train_generator(model1, batch_train_iter_doc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
235/234 [==============================] - 678s 3s/step - loss: 0.9246 - categorical_accuracy: 0.5665
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,categorical_accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


235/234 [==============================] - 657s 3s/step - loss: 0.7757 - categorical_accuracy: 0.6548
Epoch 3/10
235/234 [==============================] - 658s 3s/step - loss: 0.7503 - categorical_accuracy: 0.6644
Epoch 4/10
235/234 [==============================] - 661s 3s/step - loss: 0.7370 - categorical_accuracy: 0.6649
Epoch 5/10
235/234 [==============================] - 671s 3s/step - loss: 0.6562 - categorical_accuracy: 0.7168
Epoch 6/10
235/234 [==============================] - 677s 3s/step - loss: 0.6788 - categorical_accuracy: 0.7023
Epoch 7/10
235/234 [==============================] - 672s 3s/step - loss: 0.6737 - categorical_accuracy: 0.7055
Epoch 8/10
235/234 [==============================] - 674s 3s/step - loss: 0.6622 - categorical_accuracy: 0.7137
Epoch 9/10
235/234 [==============================] - 671s 3s/step - loss: 0.6381 - categorical_accuracy: 0.7328
Epoch 10/10
235/234 [==============================] - 668s 3s/step - loss: 0.6256 - categorical_accuracy: 

##Train on aspect-level data

In [ ]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size)

import keras.optimizers as opt
optimizer = opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
#optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_generator(model, batch_train_iter_aspect, batch_val_iter_aspect):
    
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]
    
    def train_gen():
        while True:
            train_batches = [[[X_aspect, aspect_terms], [y_aspect]] for X_aspect, \
                             y_aspect, aspect_terms in batch_train_iter_aspect]
            for train_batch in train_batches:
                yield train_batch
    def val_gen():
        while True:
            val_batches = [[[X_aspect, aspect_terms], [y_aspect]] for X_aspect, \
                           y_aspect, aspect_terms in batch_val_iter_aspect]
            for val_batch in val_batches:
                yield val_batch

    history = model.fit_generator(train_gen(), validation_data=val_gen(), \
                                  validation_steps=val_steps_epoch, steps_per_epoch=train_steps_epoch, \
                                  epochs = 10, callbacks = earlystop_callbacks)

In [ ]:
train_generator(model2, batch_train_iter_aspect, batch_val_iter_aspect)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
15/14 [===============================] - 15s 991ms/step - loss: 0.8961 - categorical_accuracy: 0.6083 - val_loss: 0.7422 - val_categorical_accuracy: 0.6406
Epoch 2/10
15/14 [===============================] - 13s 881ms/step - loss: 0.7953 - categorical_accuracy: 0.6542 - val_loss: 1.0118 - val_categorical_accuracy: 0.5859
Epoch 3/10
15/14 [===============================] - 13s 883ms/step - loss: 0.7821 - categorical_accuracy: 0.6542 - val_loss: 0.8394 - val_categorical_accuracy: 0.6328
Epoch 4/10
15/14 [===============================] - 13s 889ms/step - loss: 0.7933 - categorical_accuracy: 0.6396 - val_loss: 0.9712 - val_categorical_accuracy: 0.6641
Epoch 5/10
15/14 [===============================] - 13s 884ms/step - loss: 0.6863 - categorical_accuracy: 0.7021 - val_loss: 0.6697 - val_categorical_accuracy: 0.6484
Epoch 6/10
15/14 [===============================] - 13s 872ms/step - loss: 0.6498 - categorical_accuracy: 0.7125 - val_loss: 0.6468 - val_categorical_accuracy:

##Evaluating on test set
- show the accuracy

In [ ]:
test_loss, test_acc = model2.evaluate([test_x, test_aspect], [test_y], verbose=0)
print('Test loss: %.4f, acc: %.4f' % (test_loss, test_acc))


Test loss: 0.8120, acc: 0.6285
